<a href="https://colab.research.google.com/github/lionelsaxer/DLSS/blob/main/Final_Essay_DLSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Essay DLSS Code

In [66]:
# Import Packages
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from collections import Counter
from operator import itemgetter

In [67]:
# Import data set
df_vdem = pd.read_csv("V-Dem-CY-Full+Others-v13.csv",
                      usecols=["v2eltrnout", "country_name", "year", "v2elvaptrn", "e_wb_pop", "v2elcomvot", "v2elpubfin",
                               "v2ellovtlg", "v2ellovtsm", "v2elparlel"])

In [68]:
df_vdem = df_vdem[["v2eltrnout", "country_name", "year", "v2elvaptrn",
                    "e_wb_pop", "v2elcomvot", "v2elpubfin", "v2ellovtlg",
                    "v2ellovtsm", "v2elparlel"]]

In [69]:
df_vdem.head()

,v2eltrnout,country_name,year,v2elvaptrn,e_wb_pop,v2elcomvot,v2elpubfin,v2ellovtlg,v2ellovtsm,v2elparlel
0,NaN,Mexico,1789,NaN,NaN,NaN,-2.176,NaN,NaN,NaN
1,NaN,Mexico,1790,NaN,NaN,NaN,-2.176,NaN,NaN,NaN
2,NaN,Mexico,1791,NaN,NaN,NaN,-2.176,NaN,NaN,NaN
3,NaN,Mexico,1792,NaN,NaN,NaN,-2.176,NaN,NaN,NaN
4,NaN,Mexico,1793,NaN,NaN,NaN,-2.176,NaN,NaN,NaN


In [70]:
# Look at data for Switzerland
df_vdem_Switzerland = df_vdem.loc[df_vdem["country_name"]=="Switzerland"]
df_vdem_Switzerland = df_vdem_Switzerland.dropna()

In [71]:
df_vdem_Switzerland

,v2eltrnout,country_name,year,v2elvaptrn,e_wb_pop,v2elcomvot,v2elpubfin,v2ellovtlg,v2ellovtsm,v2elparlel
756,64.46,Switzerland,1963,53.46,5694247.0,0.0,-2.215,26.60,23.9,1.0
760,63.76,Switzerland,1967,53.21,5991785.0,0.0,-2.215,23.50,23.2,1.0
764,56.36,Switzerland,1971,45.02,6213399.0,0.0,-2.215,22.90,21.7,1.0
768,52.39,Switzerland,1975,43.62,6338632.0,0.0,-2.215,24.90,22.2,1.0
772,48.06,Switzerland,1979,40.60,6294365.0,0.0,-2.215,24.40,24.0,1.0
776,48.91,Switzerland,1983,40.79,6418773.0,0.0,-2.215,23.30,22.8,1.0
780,47.48,Switzerland,1987,39.90,6545106.0,0.0,-2.215,22.90,19.6,1.0
784,46.04,Switzerland,1991,39.67,6799978.0,0.0,-2.215,21.00,18.5,1.0
788,42.25,Switzerland,1995,35.66,7040687.0,0.0,-1.979,21.80,20.2,1.0
792,43.22,Switzerland,1999,34.94,7143991.0,0.0,-1.979,22.60,22.5,1.0


In [72]:
# Pre-processing
df_vdem = df_vdem.dropna() # remove missing values
c = Counter(df_vdem.country_name)
min_key, min_count = min(c.items(), key=itemgetter(1))
print(min_key, min_count)

Haiti 1


In [73]:
def merge(data1, data2):
  x1, y1 = data1
  x2, y2 = data2
  return tf.concat([x1, x2], axis=-1), tf.concat([y1, y2], axis=-1)

In [74]:
k=1
for country in df_vdem.country_name.unique():
  df_h=df_vdem[df_vdem.country_name==country]
  if df_h.shape[0]>10:
    # remove country name, year and VAP turnout
    print(country)
    df_h=df_h[["v2eltrnout", "e_wb_pop", "v2elcomvot",\
        "v2elpubfin", "v2elparlel", "v2ellovtlg", "v2ellovtsm"]]

    raw_data = df_h.to_numpy()
    num_train_samples = int(0.5 * len(raw_data))
    num_val_samples = int(0.25 * len(raw_data))
    num_test_samples = len(raw_data) - num_train_samples - num_val_samples

    # normalize data
    mean = raw_data[:num_train_samples].mean(axis=0)
    raw_data -= mean
    std = raw_data[:num_train_samples].std(axis=0)
    raw_data /= (std + 1e-9)

    sampling_rate = 1 # sample every data point
    sequence_length = 3 # use last 3 elections to predict
    delay = sampling_rate * (sequence_length - 1)
    batch_size = 3
    # index to predict
    targets0=pd.DataFrame(raw_data)[0].to_numpy()
  try:
    if k==1:
      train_dataset = keras.utils.timeseries_dataset_from_array(
          raw_data[:-delay],
          targets=targets0[delay:],
          sampling_rate=sampling_rate,
          sequence_length=sequence_length,
          shuffle=True,
          batch_size=batch_size,
          start_index=0,
          end_index=num_train_samples)

      val_dataset = keras.utils.timeseries_dataset_from_array(
          raw_data[:-delay],
          targets=targets0[delay:],
          sampling_rate=sampling_rate,
          sequence_length=sequence_length,
          shuffle=True,
          batch_size=batch_size,
          start_index=num_train_samples,
          end_index=num_train_samples + num_val_samples - 1)

      test_dataset = keras.utils.timeseries_dataset_from_array(
          raw_data[:-delay],
          targets=targets0[delay:],
          sampling_rate=sampling_rate,
          sequence_length=sequence_length,
          shuffle=True,
          batch_size=batch_size,
          start_index=num_train_samples + num_val_samples - 1)
    else:
        train_dataset2 = keras.utils.timeseries_dataset_from_array(
          raw_data[:-delay],
          targets=targets0[delay:],
          sampling_rate=sampling_rate,
          sequence_length=sequence_length,
          shuffle=True,
          batch_size=batch_size,
          start_index=0,
          end_index=num_train_samples)

        val_dataset2 = keras.utils.timeseries_dataset_from_array(
            raw_data[:-delay],
            targets=targets0[delay:],
            sampling_rate=sampling_rate,
            sequence_length=sequence_length,
            shuffle=True,
            batch_size=batch_size,
            start_index=num_train_samples,
            end_index=num_train_samples + num_val_samples - 1)

        test_dataset2 = keras.utils.timeseries_dataset_from_array(
            raw_data[:-delay],
            targets=targets0[delay:],
            sampling_rate=sampling_rate,
            sequence_length=sequence_length,
            shuffle=True,
            batch_size=batch_size,
            start_index=num_train_samples + num_val_samples - 1)
        k=2


        train_dataset = tf.data.Dataset.zip((train_dataset, train_dataset2)).map(merge)
        val_dataset = tf.data.Dataset.zip((val_dataset, val_dataset2)).map(merge)
        test_dataset = tf.data.Dataset.zip((test_dataset, test_dataset2)).map(merge)

  except:
    k=k+1

Mexico
Sweden
Switzerland
Japan
Colombia
Brazil
United States of America
Portugal
El Salvador
Bolivia
Peru
Argentina
India
Philippines
Thailand
Venezuela
Indonesia
Canada
Australia
Botswana
Chile
Costa Rica
Ecuador
France
Germany
Guatemala
Ireland
Italy
Netherlands
Spain
Turkey
United Kingdom
Uruguay
Dominican Republic
Jamaica
Sri Lanka
Trinidad and Tobago
Austria
Barbados
Belgium
Cyprus
Denmark
Finland
Greece
Guyana
Iceland
Israel
Luxembourg
Malta
Mauritius
New Zealand
Norway
Paraguay
Singapore


In [75]:
for x, y in test_dataset:
  print(x.shape, y.shape)

(1, 3, 7) (1,)


In [76]:
# Estimate a simple, densely connected baseline
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.LSTM(16)(inputs)
outputs = layers.Dense(1)(x) # predicting turnout
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("vdem_baseline.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=2,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("vdem_baseline.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/2
1/2 [==============>...............] - ETA: 4s - loss: 0.2276 - mae: 0.4691

2/2 [==============================] - 4s 44ms/step - loss: 0.2411 - mae: 0.4845
Epoch 2/2
1/2 [==============>...............] - ETA: 0s - loss: 0.1915 - mae: 0.4345

2/2 [==============================] - 0s 26ms/step - loss: 0.2074 - mae: 0.4522


OSError: ignored

In [ ]:
model.summary()

In [ ]:
# Plot MAE for training and validation data
loss = history.history["mae"]
val_loss = history.history["val_mae"]
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, "bo", label="Training MAE")
plt.plot(epochs, val_loss, "b", label="Validation MAE")
plt.title("Training and validation MAE")
plt.legend()
plt.show()

In [ ]:
# Dropout-regularized LSTM
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.LSTM(32, recurrent_dropout=0.25)(inputs)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("jena_lstm_dropout.keras",
                                    save_best_only=True)
]
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=2,
                    validation_data=val_dataset,
                    callbacks=callbacks)